In [2]:
import flax
import flax.linen as flax_nn
from numpyro.contrib.module import flax_module
import numpy as np
from jax import nn
import jax.numpy as jnp
import numpyro 
import numpyro.distributions as dist
import numpyro.distributions.constraints as constraints

assert numpyro.__version__.startswith("0.9.2")

In [10]:
def model(data):
    S = 12
    N = 6
    #set hyperparameters
    alpha_p = 1
    beta_p = 1
    alpha_pi= 1
    beta_pi = 5

    #global variables
    p = numpyro.sample('p', dist.Beta(alpha_p, beta_p),sample_shape=(1,1))
    print(p)

    with numpyro.plate("pi_plate", N):
        pi = numpyro.sample('pi', dist.Beta(alpha_pi, beta_pi))

    with numpyro.plate('r_plate', S):
        r = numpyro.sample('r', dist.Bernoulli(probs=p))
    
  
    print(pi)
    with   numpyro.plate("r_plate", S), numpyro.plate("pi_plate", N):
        # r_pi = torch.transpose(torch.unsqueeze(r, 0), 0, 1)

        # t_pi = pi
        t_pi = torch.transpose(torch.unsqueeze(pi, 0) , 0, 1)
        # print(t_pi.size())
        # print("t_pi:{}, t_pi.size={}".format(t_pi,t_pi.size())) 
        # print("r*t_pi.size={}, t_pi*r.size={}, r*t_pi={}".format((r*t_pi).size(),(t_pi*r).size(),(r*t_pi)))
        # mu = numpyro.sample("mu", dist.Bernoulli( r_pi * ( 1 - pi) + ( 1 - r_pi ) * pi ), obs=data)
        mu = numpyro.sample("mu", dist.Bernoulli( r * ( 1 - t_pi) + ( 1 - r ) * t_pi ), obs=data)
        
    return p, r,pi,mu


In [11]:
data = jnp.ones(12)
# data = torch.tensor([])
print("data:",data)
p,r,pi ,mu = model(data)
print(mu)
print(pi)
# print(mu)
numpyro.render_model(model, model_args=(data,))

data: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


AssertionError: 